<a href="https://colab.research.google.com/github/saraHuang/LLM_study/blob/main/LangChain_%E7%AF%87_%E9%95%B7%E6%96%87%E6%91%98%E8%A6%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🥱 LangChain 篇_長文摘要

❤️ Created by [hengshiousheu](https://huggingface.co/Heng666).

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/summarization_use_case_1.png?raw=1)

## 整體概念

本筆記本主要是教授第一次使用 LangChain 使用者，如何搭建一個長文章摘要工具。

建構摘要器的核心問題是如何將您的文件傳遞到 LLM 的內容視窗。以下兩種常見方法：

1. `Stuff` ：只需將所有文件「填充」到單一提示中。這是最簡單的方法。

2. `Map-reduce` ：在「對應」步驟中分別摘要每個文件，然後將摘要「簡化」成最終摘要


![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/summarization_use_case_2.png?raw=1)

## 快速入門操作篇

為了讓您先睹為快，只要使用 LangChain 提供的 Chain 即可：`load_summarize_chain`。

假設我們想要摘要一篇部落格文章。我們可以使用幾行程式碼來建立它。

首先設定環境變數並安裝套件：

##環境建置

In [ ]:
%pip install --upgrade --quiet  langchain-openai tiktoken chromadb langchain langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.7 MB/s eta 0:00:

我們需要設置 環境變數 `OPENAI_API_KEY` ，可以直接完成，如下所示：

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


我們可以使用 `chain_type="stuff"` ，尤其是處理較大的文件時，同時模型要選大窗口的，例如：

* 16k token OpenAI `gpt-3.5-turbo-1106`
* 100k token Anthropic [Claude-2](https://www.anthropic.com/index/claude-2)

我們也可以提供 `chain_type="map_reduce"` 或 `chain_type="refine"` 策略進行長文摘要。

更多資訊 [LangChain Chain Docs](https://python.langchain.com/docs/modules/chains)

In [ ]:
!pip install --quiet langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

#擷取文章
loader = WebBaseLoader("https://www.fcu.edu.tw/event/?id=B0790BF52E78459F86841E241EB86ACD")
docs = loader.load()

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
chain = load_summarize_chain(llm, chain_type="stuff")

sum_result = chain.run(docs)

print(sum_result)

/tmp/ipython-input-5-2623059938.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  sum_result = chain.run(docs)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: https://*************************************************************************ring. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## 摘要策略ㄧ、 填充(Stuff)

當我們將 `load_summarize_chain` 與 `chain_type="stuff"` 一起使用時，我們將使用 StuffDocumentsChain。

Chain 將採用文件清單，將它們全部插入提示中，然後將該提示傳遞給 LLM：

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary of the following In TRADTIONAL CHINESE:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
print(stuff_chain.run(docs))

【人文與科技大講堂】舞蹈家許芳宜 - 變形記 | 逢甲大學
逢甲大學舉辦了一場名為【人文與科技大講堂】的講座，邀請到舞蹈家許芳宜以主題「變形記」進行演講。許芳宜是美國現代舞之母瑪莎葛蘭姆的傳人，也是國家文藝獎得主和金馬獎最佳動作設計獎得主。講座將於113年3月12日在逢甲大學舉行。


太棒了！我們可以看到我們使用 `load_summarize_chain` 重現了較早的結果。

### 嘗試更多？

* 您可以輕鬆自訂提示
* 您可以透過 `llm` 參數輕鬆嘗試不同的 LLMs（例如，Claude）。

## 摘要策略二、拆分與整合(Map-Reduce)

讓我們解構 map reduce 方法。為此，我們將首先使用 `LLMChain` 將每個文件映射到個別摘要。然後，我們將使用 `ReduceDocumentsChain` 將這些摘要合併成單一全局摘要。

首先，我們指定用於將每個文件映射到個別摘要的 LLMChain：

In [ ]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import CharacterTextSplitter

llm = ChatOpenAI(temperature=0)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

我們也可以使用提示中心來儲存和擷取提示。


In [ ]:
from langchain import hub

map_prompt = hub.pull("rlm/map-prompt")
map_chain = LLMChain(llm=llm, prompt=map_prompt)

`ReduceDocumentsChain` 負責取得文件映射結果並將其簡化成單一輸出。它包裝一個通用 `CombineDocumentsChain` （例如 `StuffDocumentsChain` ），但增加了在將文件傳遞給 CombineDocumentsChain 之前將其壓縮的功能，如果其累積大小超過 `token_max` 。在此範例中，我們實際上可以重複使用我們的鏈來組合文件，以壓縮文件。

因此，如果我們映射文件中的累計權杖數超過 4000 個權杖，那麼我們將遞迴地將文件分批傳遞給權杖數小於 4000 的 `StuffDocumentsChain` 以建立批次摘要。一旦這些批次摘要累計小於 4000 個權杖，我們將最後一次將它們全部傳遞給 `StuffDocumentsChain` 以建立最終摘要。

In [ ]:
# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [ ]:
# Note we can also get this from the prompt hub, as noted above
reduce_prompt = hub.pull("rlm/map-prompt")

In [ ]:
reduce_prompt

ChatPromptTemplate(input_variables=['docs'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], template='The following is a set of documents:\n{docs}\nBased on this list of docs, please identify the main themes \nHelpful Answer:'))])

In [ ]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

將我們的拆分和總結鏈組合成一個：

In [ ]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [ ]:
print(map_reduce_chain.run(split_docs))

The main themes of the documents provided include academic information and fees, campus organization and facilities, transportation information, enrollment information for local and international students, continuing education and services for high school students, events and activities on campus, guest lectures and cultural events, subscription to newsletters and updates, general information and homepage, contact information and communication channels, public information on school affairs and finances, sustainability report, and gender equality education committee. These themes cover a wide range of topics related to education, student life, administrative services, and campus events at Feng Chia University.


### 想了解更多？

**客製化環節**

* 您可以自訂 LLMs 以及映射和簡化階段的提示。

**現實世界案例分享**

* 可以參考[部落格文章](https://blog.langchain.dev/llms-to-improve-documentation/)
* 部落格文章和相關[儲存庫](https://github.com/mendableai/QA_clustering)還介紹了將叢集作為摘要方法。

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/summarization_use_case_3.png?raw=1)